In [61]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import gmaps
import plotly.graph_objects as go
import plotly.express as px
# Google developer API key
from config import gkey
from ipywidgets.embed import embed_minimal_html
# Configure gmaps
gmaps.configure(api_key=gkey)
# Load in csv
university_data= ("Most-Recent-Cohorts-All-Data-Elements.csv")

university_df = pd.read_csv(university_data, low_memory=False)
reduced_university_df =university_df[["UNITID","INSTNM","CITY","STABBR","ZIP","ACCREDAGENCY",
                                     "INSTURL","LATITUDE","LONGITUDE","MN_EARN_WNE_P6","MN_EARN_WNE_P10"]]
reduced_university_df = reduced_university_df.rename(
    columns={"UNITID":"University ID","INSTNM":"University Name","CITY":"City","STABBR":"State","ZIP":"Zip",
                                     "INSTURL":"Website","LATITUDE":"Latitude","LONGITUDE":"Longitude"})


reduced_university_df.head(5)



,University ID,University Name,City,State,Zip,ACCREDAGENCY,Website,Latitude,Longitude,MN_EARN_WNE_P6,MN_EARN_WNE_P10
0,100654,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,34.783368,-86.568502,28400,35500
1,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,33.505697,-86.799345,39400,48400
2,100690,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,32.362609,-86.174010,35400,47600
3,100706,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,34.724557,-86.640449,40300,52000
4,100724,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,32.364317,-86.295677,24400,30600


In [2]:
#checking the length of the dataset
len(reduced_university_df)

7112

In [3]:
###viewing number of rows with null values in each column
reduced_university_df.isna().sum()

University ID         0
University Name       0
City                  0
State                 0
Zip                   0
ACCREDAGENCY        563
Website              19
Latitude            444
Longitude           444
MN_EARN_WNE_P6     1225
MN_EARN_WNE_P10    1225
dtype: int64

In [4]:
####dropping rows with null values
reduced_university_df = reduced_university_df.dropna(how='any')

In [5]:
###confirming that rows with null values have been removed
reduced_university_df.isna().sum()

University ID      0
University Name    0
City               0
State              0
Zip                0
ACCREDAGENCY       0
Website            0
Latitude           0
Longitude          0
MN_EARN_WNE_P6     0
MN_EARN_WNE_P10    0
dtype: int64

In [6]:
###find number of unique univsities
len(reduced_university_df["University ID"].unique())

5176

In [7]:
reduced_university_df = reduced_university_df.set_index("MN_EARN_WNE_P6")

reduced_university_df.head()

,University ID,University Name,City,State,Zip,ACCREDAGENCY,Website,Latitude,Longitude,MN_EARN_WNE_P10
MN_EARN_WNE_P6,,,,,,,,,,
28400,100654,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,34.783368,-86.568502,35500
39400,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,33.505697,-86.799345,48400
35400,100690,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,32.362609,-86.174010,47600
40300,100706,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,34.724557,-86.640449,52000
24400,100724,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,32.364317,-86.295677,30600


In [8]:
####drop rows with privacysuppressed
reduced_university_df = reduced_university_df.drop("PrivacySuppressed", axis=0)



In [9]:
###confirm privacysuppressed rows were dropped
len(reduced_university_df["University ID"].unique())

4695

In [10]:
####reset index
reduced_university_df = reduced_university_df.reset_index()

reduced_university_df.head()

,MN_EARN_WNE_P6,University ID,University Name,City,State,Zip,ACCREDAGENCY,Website,Latitude,Longitude,MN_EARN_WNE_P10
0,28400,100654,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,34.783368,-86.568502,35500
1,39400,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,33.505697,-86.799345,48400
2,35400,100690,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,32.362609,-86.174010,47600
3,40300,100706,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,34.724557,-86.640449,52000
4,24400,100724,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,32.364317,-86.295677,30600


In [11]:
reduced_university_df = reduced_university_df.set_index("MN_EARN_WNE_P10")

In [12]:
reduced_university_df.head()

,MN_EARN_WNE_P6,University ID,University Name,City,State,Zip,ACCREDAGENCY,Website,Latitude,Longitude
MN_EARN_WNE_P10,,,,,,,,,,
35500,28400,100654,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,34.783368,-86.568502
48400,39400,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,33.505697,-86.799345
47600,35400,100690,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,32.362609,-86.174010
52000,40300,100706,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,34.724557,-86.640449
30600,24400,100724,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,32.364317,-86.295677


In [13]:
reduced_university_df = reduced_university_df.drop("PrivacySuppressed", axis=0)

In [14]:
reduced_university_df = reduced_university_df.reset_index()
reduced_university_df.head()

,MN_EARN_WNE_P10,MN_EARN_WNE_P6,University ID,University Name,City,State,Zip,ACCREDAGENCY,Website,Latitude,Longitude
0,35500,28400,100654,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,34.783368,-86.568502
1,48400,39400,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,33.505697,-86.799345
2,47600,35400,100690,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,32.362609,-86.174010
3,52000,40300,100706,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,34.724557,-86.640449
4,30600,24400,100724,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,32.364317,-86.295677


In [15]:
###confirm privacysuppressed rows were dropped
reduced_university_df.count()

MN_EARN_WNE_P10    4441
MN_EARN_WNE_P6     4441
University ID      4441
University Name    4441
City               4441
State              4441
Zip                4441
ACCREDAGENCY       4441
Website            4441
Latitude           4441
Longitude          4441
dtype: int64

In [16]:
reduced_university_df.dtypes

MN_EARN_WNE_P10     object
MN_EARN_WNE_P6      object
University ID        int64
University Name     object
City                object
State               object
Zip                 object
ACCREDAGENCY        object
Website             object
Latitude           float64
Longitude          float64
dtype: object

In [17]:
reduced_university_df['MN_EARN_WNE_P10'] = pd.to_numeric(reduced_university_df['MN_EARN_WNE_P10'])

In [18]:
reduced_university_df.dtypes

MN_EARN_WNE_P10      int64
MN_EARN_WNE_P6      object
University ID        int64
University Name     object
City                object
State               object
Zip                 object
ACCREDAGENCY        object
Website             object
Latitude           float64
Longitude          float64
dtype: object

In [19]:
reduced_university_df['MN_EARN_WNE_P6'] = pd.to_numeric(reduced_university_df['MN_EARN_WNE_P6'])

In [20]:
reduced_university_df.dtypes

MN_EARN_WNE_P10      int64
MN_EARN_WNE_P6       int64
University ID        int64
University Name     object
City                object
State               object
Zip                 object
ACCREDAGENCY        object
Website             object
Latitude           float64
Longitude          float64
dtype: object

In [21]:
####calculating percent change
reduced_university_df["Percent Change"]=((reduced_university_df["MN_EARN_WNE_P10"]-reduced_university_df["MN_EARN_WNE_P6"])/reduced_university_df["MN_EARN_WNE_P6"])*100
reduced_university_df.head()

,MN_EARN_WNE_P10,MN_EARN_WNE_P6,University ID,University Name,City,State,Zip,ACCREDAGENCY,Website,Latitude,Longitude,Percent Change
0,35500,28400,100654,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,34.783368,-86.568502,25.000000
1,48400,39400,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,33.505697,-86.799345,22.842640
2,47600,35400,100690,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,32.362609,-86.174010,34.463277
3,52000,40300,100706,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,34.724557,-86.640449,29.032258
4,30600,24400,100724,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,32.364317,-86.295677,25.409836


In [22]:
# reduced_university_df["Percent Change"] = reduced_university_df["Percent Change"].astype(float).map("{:,.2f}%".format)
reduced_university_df.head()

,MN_EARN_WNE_P10,MN_EARN_WNE_P6,University ID,University Name,City,State,Zip,ACCREDAGENCY,Website,Latitude,Longitude,Percent Change
0,35500,28400,100654,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,34.783368,-86.568502,25.000000
1,48400,39400,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,33.505697,-86.799345,22.842640
2,47600,35400,100690,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,32.362609,-86.174010,34.463277
3,52000,40300,100706,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,34.724557,-86.640449,29.032258
4,30600,24400,100724,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,32.364317,-86.295677,25.409836


In [46]:
university_sorted=reduced_university_df.sort_values((["MN_EARN_WNE_P10"]),ascending=False)
Top_50_mean= university_sorted.head(50)


In [47]:
####check that the df is only top 50

Top_50_mean.dtypes

MN_EARN_WNE_P10      int64
MN_EARN_WNE_P6       int64
University ID        int64
University Name     object
City                object
State               object
Zip                 object
ACCREDAGENCY        object
Website             object
Latitude           float64
Longitude          float64
Percent Change     float64
dtype: object

In [49]:


Top_10_mean =Top_50_mean.head(10)
len(Top_10_mean)

10

In [51]:
# Store latitude and longitude in locations
locations = Top_50_mean[["Latitude", "Longitude"]]
Percent_change = Top_50_mean["Percent Change"].tolist()
University_name = Top_50_mean["University Name"].tolist()
# Fill NaN values and convert to float
rating =Top_50_mean["MN_EARN_WNE_P10"].astype(float)

In [52]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=rating)
                                 
                                 
#                                  , weights=rating, 
#                                  dissipating=True, max_intensity=10,
#                                  point_radius=1)


# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [62]:
percent_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Percentage Change: {percent}" for percent in Percent_change]
)
heat_layer = gmaps.heatmap_layer(locations,weights=rating)

University_layer = gmaps.symbol_layer(
    locations, scale=2,
    info_box_content=[f"University Name: {uni}" for uni in University_name]
)

# fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', 
fig = gmaps.figure()
fig.add_layer(University_layer)
fig.add_layer(heat_layer)
fig
embed_minimal_html('export.html', views=[fig])

In [35]:

grouped_university= reduced_university_df.groupby('State').agg({'MN_EARN_WNE_P10':['mean']})
# Find how many rows fall into each bin
# grouped_university["Percent Change"].mean()
# grouped_university=reduced_university_df.sort_values((["Percent Change"]),ascending=False)
# len(grouped_university)

grouped_university=pd.DataFrame(grouped_university).reset_index()
grouped_university
grouped_university.columns = grouped_university.columns.droplevel(1)

grouped_university_sorted=grouped_university.sort_values((["MN_EARN_WNE_P10"]),ascending=False)

In [36]:
grouped_university_sorted_10 = grouped_university_sorted.head(10)
grouped_university_sorted_10.head()

,State,MN_EARN_WNE_P10
8,DC,55078.571429
22,MA,52434.453782
44,RI,49461.111111
38,NY,48930.208333
7,CT,47676.785714


In [66]:
bar_plot=px.bar(grouped_university_sorted_10, y='State', x='MN_EARN_WNE_P10', orientation='h',color='MN_EARN_WNE_P10',
           labels={'MN_EARN_WNE_P10':'Mean Earnings', 'State':'States'}, color_continuous_scale='Blues')
#making layout changes
bar_plot.update_layout(xaxis_tickangle=-45, title_text='Top 10 States for Highest Mean Earnings After 10 Years')
#outputing plot
bar_plot.show()

In [ ]:
# #using scatter geo with above created subdataframe
# fig = px.scatter_geo(data_frame=grouped_university_sorted_10, scope='south america',lat='Lat',lon='Long',
#                      size='Count', color='State', projection='hammer')
# fig.update_layout(
#         title_text = '1998-2017 Top-10 States in Brazil with reported fires')
# fig.show()

# grouped_university_2= reduced_university_df.groupby('State').agg({'MN_EARN_WNE_P10':['mean'],'Latitude','Longitude'})

In [54]:
# Load in csv
state_data= ("statelatlong.csv")

state_df = pd.read_csv(state_data, low_memory=False)
state_df.head()

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [55]:
merged_state_latlong =pd.merge(state_df, grouped_university_sorted_10, on="State")

In [56]:
merged_state_latlong.head(10)

,State,Latitude,Longitude,City,MN_EARN_WNE_P10
0,CA,37.271875,-119.270415,California,43150.529101
1,CT,41.518784,-72.757507,Connecticut,47676.785714
2,DC,38.899349,-77.014567,District of Columbia,55078.571429
3,MD,38.806352,-77.268416,Maryland,44003.333333
4,MA,42.062940,-71.718067,Massachusetts,52434.453782
5,NE,41.500819,-99.680902,Nebraska,43483.783784
6,NY,40.705626,-73.979680,New York,48930.208333
7,PA,40.994593,-77.604698,Pennsylvania,45589.694656
8,RI,41.582728,-71.506451,Rhode Island,49461.111111
9,WA,38.899349,-77.014567,Washington,43142.666667


In [57]:
del merged_state_latlong['City']

In [58]:
merged_state_latlong.head(10)
# merged_state_latlong.columns = ['State', 'Latitude']

,State,Latitude,Longitude,MN_EARN_WNE_P10
0,CA,37.271875,-119.270415,43150.529101
1,CT,41.518784,-72.757507,47676.785714
2,DC,38.899349,-77.014567,55078.571429
3,MD,38.806352,-77.268416,44003.333333
4,MA,42.062940,-71.718067,52434.453782
5,NE,41.500819,-99.680902,43483.783784
6,NY,40.705626,-73.979680,48930.208333
7,PA,40.994593,-77.604698,45589.694656
8,RI,41.582728,-71.506451,49461.111111
9,WA,38.899349,-77.014567,43142.666667


In [ ]:
# fig = px.choropleth(merged_state_latlong, locations="iso_alpha" , color="MN_EARN_WNE_P10", hover_name="State",range_color=[20,80])
# fig.show()

In [59]:
#using scatter geo with above created subdataframe
fig = px.scatter_geo(data_frame=merged_state_latlong, lat='Latitude',scope='north america',lon='Longitude',
                     size='MN_EARN_WNE_P10', color='State', projection='natural earth')
fig.update_layout(
        title_text = 'Top 10 States with Highest Mean Earnings 10 Years After Graduation')
fig.show()
# animation_frame="State"